In [1]:
import pandas as pd
import datetime
import pyupbit

import math
import time

In [2]:
# utils

def make_levels(mode, upper, lower, grids):
    # grid level 생성
    lower = pyupbit.get_tick_size(lower)
    upper = pyupbit.get_tick_size(upper)
    levels = [lower]
    cnt = 1
    # 동일한 가격 차이로 levels 생성
    if mode == "Arithmetic":
        while cnt < grids:
            diff =  (upper - lower)/(grids) # 주문들 사이의 가격 차이
            price = lower + diff * cnt
            price = pyupbit.get_tick_size(price) # 업비트 호가에 맞춰서 조정
            levels.append(price)
            cnt += 1
    # 동일한 가격 비율로 levels 생성
    elif mode == "Geometric":
        while cnt < grids:
            diff = (upper / lower) ** (1/(grids)) # 주문들 사이의 가격 비율 차이
            price = lower * (diff ** cnt)
            price = pyupbit.get_tick_size(price) # 업비트 호가에 맞춰서 조정
            levels.append(price)
            cnt += 1
            
    # # 마지막에 UPPER 값 넣기
    # levels.append(upper)
    
    return levels, diff

# 호가 단위 함수
def get_price_scale_tick(_price):   
    if _price >= 2000000: 
        return -3, 1000
    elif _price >= 1000000: 
        return -2, 500
    elif _price >= 500000: 
        return -2, 100
    elif _price >= 100000: 
        return -1, 50
    elif _price >= 10000: 
        return -1, 10
    elif _price >= 1000: 
        return 0, 5
    elif _price >= 100: 
        return 0, 1
    elif _price >= 10: 
        return 1, 0.1
    elif _price >= 0: 
        return 2, 0.01

In [11]:
# upbit 연결
with open("upbit_key.txt", 'r') as f:
    lines = f.read().split('\n')
    access_key = lines[0]
    secret_key = lines[1]
upbit = pyupbit.Upbit(access_key, secret_key)

# 지정 파라미터
INTERVAL = 30 # 변동성 계산할 때 몇분 봉 사용
STD_NUM = 40 # 변동성 계산할 때 과거 몇개 사용
LOWER_STD = 2 # 하한선 변동성 배수
MODE = "Arithmetic" # Arithmetic: 동일한 가격 차이 / Geometric: 동일한 가격 비율 차이
GRIDS = 6 # 그리드 수
COIN = "BTC" # 거래할 코인
BUDGET = 80000 # 사용할 예산

STOP_LOSS = 10 # 손절매 퍼센트
RESET_GRID = 5 # 계속 상승할 경우 그리드 리셋하는 퍼센트

In [12]:
# 현재가 조회
current = pyupbit.get_current_price(f"KRW-{COIN}")

# 잔고 조회
balance_KRW = upbit.get_balance("KRW")
balance_coin = upbit.get_balance(COIN)
if balance_coin != 0:
    raise Exception("코인 수가 0이 아닙니다.")

print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
print(f"{COIN} 현재가(KRW):",current)

# 호가 단위
_, scale = get_price_scale_tick(current)
print("호가단위(KRW):", scale)

# 과거 데이터 조회해서 변동성 계산
df = pyupbit.get_ohlcv(f"KRW-{COIN}", interval=f"minute{INTERVAL}")
std = df.iloc[-STD_NUM:].close.std()
print(f"{INTERVAL}분봉 {STD_NUM}개 변동성: {std:.2f}")

# 상한선 하한선 계산|
upper = current 
lower = current - LOWER_STD*std
if current < upper:
    raise Exception("상한선은 현재가보다 낮아야합니다.")

# grid levels 생성
levels, diff = make_levels(MODE, upper, lower, GRIDS)
print("Levels:", levels)
print(f"Difference(KRW): {diff:.4f}")

# 주문량 생성
volume = math.floor((BUDGET / GRIDS)/current*1e8)/1e8 # 업비트는 소수 8자리까지 주문 가능
if volume * current < 5000: # 최소 주문 단위 5000보다 작을 경우 error
    raise Exception(f"최소 주문 단위보다 작습니다. {volume * current:.2f}원")
print(f"Volume({COIN}):", volume)

print(f"수익률: {diff/current*100:.2f}%")

2022-06-21 14:18:39
BTC 현재가(KRW): 26773000.0
호가단위(KRW): 1000
30분봉 40개 변동성: 259962.00
Levels: [26253000, 26339000, 26426000, 26513000, 26599000, 26686000]
Difference(KRW): 86666.6667
Volume(BTC): 0.00049801
수익률: 0.32%


In [7]:
# 초기 매수 주문
for price in levels:
    ret = upbit.buy_limit_order(f"KRW-{COIN}", price, volume) # 지정된 가격에 매수 주문
    time.sleep(0.1)

while True:
    time.sleep(0.2) # 0.2초마다 코인 보유량 가져오기
    now_open = len(upbit.get_order(f"KRW-{COIN}")) # 현재 미체결 주문수
    
    # 미체결 주문수가 GRIDS와 다르면 추가 주문
    if GRIDS != now_open:
        balance_coin = upbit.get_balance(COIN) # 코인 보유량으로 매수 매도 판단
        last_price = pyupbit.get_current_price(f"KRW-{COIN}") # 최근 체결 가격
        
        if balance_coin != 0: # 보유량이 0이 아니면 최근 체결은 매수
            side = "bid"
            price = last_price + diff # 매도 목표가는 diff 만큼 더한 값
            price = pyupbit.get_tick_size(price)
            ret = upbit.sell_limit_order(f"KRW-{COIN}", price, volume)
            
        else: # 보유량이 0이면 최근 체결은 매도
            side = "ask"
            price = last_price - diff # 매수 목표가는 diff 만큼 뺀 값
            price = pyupbit.get_tick_size(price)
            ret = upbit.buy_limit_order(f"KRW-{COIN}", price, volume)
        
        # 최근 체결 정보 표시
        print("")
        print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        print(f"{side} -> {last_price}KRW  {volume}{COIN} 체결")
        print(f"보유 자산(KRW) {upbit.get_balance('KRW')}")
        print(f"보유 자산({COIN}) {upbit.get_balance(COIN)}")
        # print(ret)
            
    # print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    # print(f"{COIN} 현재가(KRW):",current)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\felab\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\felab\AppData\Local\Temp/ipykernel_30016/1460561289.py", line 7, in <module>
    time.sleep(0.2) # 0.2초마다 코인 보유량 가져오기
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\felab\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\felab\appdata\local\programs\python\python37\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed

TypeError: object of type 'NoneType' has no len()

In [88]:
# STOP LOSS
avg_price = upbit.get_balances(COIN)

In [89]:
avg_price

([{'currency': 'KRW',
   'balance': '99733.47805922',
   'locked': '0.0',
   'avg_buy_price': '0',
   'avg_buy_price_modified': True,
   'unit_currency': 'KRW'}],
 {'group': 'default', 'min': 898, 'sec': 29})

In [ ]:
# 일정 이상 오르면 GRID 리셋


In [8]:
# 미체결 주문 모두 취소
order_list = upbit.get_order(f"KRW-{COIN}")

for order in order_list:
    ret_cancel = upbit.cancel_order(order["uuid"])
    time.sleep(0.1)
    
print(upbit.get_order(f"KRW-{COIN}"))

[]
